In [13]:
import mediapipe as mp
import cv2
import numpy as np

mp_drawing  = mp.solutions.drawing_utils
mp_pose = mp.solutions.holistic

In [14]:
cap = cv2.VideoCapture('ROWTRAIN.mp4')

with mp_pose.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, image = cap.read()
        
        
        #recolor
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #detections
        results = holistic.process(image)
        
        #recolor back to RGB
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                 , mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                 )
        
        
        
        
        
        cv2.imshow('RawFeed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
            
cap.release()
cv2.destroyAllWindows()

In [15]:
import csv
import os
import numpy as np

In [16]:
num_coords = len(results.pose_landmarks.landmark)

In [17]:
landmarks = ['class']
for val in range (1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [18]:
with open ('CSVtest.csv', mode='w', newline ='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [19]:
class_name = 'Catch'

In [105]:
cap = cv2.VideoCapture('CatchTrain.mp4')

with mp_pose.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, image = cap.read()
        
        
        #recolor
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #detections
        results = holistic.process(image)
        
        #recolor back to RGB
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                 , mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                 )
        
        #export coordinates

        pose = results.pose_landmarks.landmark #gets the pose landmarks
        pose_row = np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten()
            #gives each landmark component
            # .flatten() collapses an array into a single dimension, [[1,2],[3,4]] --> [1,2,3,4]

        #np.insert(pose_row, 0, class_name) #appends the class name to the CSV


            #Exports to CSV

        with open ('CSVtest.csv', 'a', newline ='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(pose_row)

            
            
            

        
        
        
        
        
        cv2.imshow('CatchFeed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
            
cap.release()
cv2.destroyAllWindows()

In [113]:
cap = cv2.VideoCapture('CatchTrain.mp4')

with mp_pose.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, image = cap.read()
        
        
        #recolor
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #detections
        results = holistic.process(image)
        
        #recolor back to RGB
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                 , mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                 )
        
        #export coordinates
        try:
            pose = results.pose_landmarks.landmark #gets the pose landmarks
            pose_row = np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten()
                #gives each landmark component
                # .flatten() collapses an array into a single dimension, [[1,2],[3,4]] --> [1,2,3,4]

      
            
        #get landmarks
            rshoulder = [pose[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,pose[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            rhip = [pose[mp_pose.PoseLandmark.RIGHT_HIP.value].x,pose[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            rknee = [pose[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,pose[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]


            angle = calculate_angle(rshoulder, rhip, rknee) #calculate angles

            cv2.putText(image, str(angle), 
                            tuple(np.multiply(rhip, [640, 480]).astype(int)), #formats coords to vid
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            
            
        except Exception as e:
            pass
        
        
        
        
        
        cv2.imshow('RowKeypoints', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
            
cap.release()
cv2.destroyAllWindows()

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [21]:
df = pd.read_csv('trainingData.csv.')

In [22]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,Catch,0.308290,0.204658,-0.033788,0.999879,0.305089,0.182503,-0.020683,0.999923,0.304762,...,-0.115396,0.943115,0.568168,0.710389,-0.004707,0.598909,0.588817,0.710681,-0.163385,0.951662
1,Catch,0.309615,0.204640,-0.043759,0.999877,0.305702,0.182495,-0.031382,0.999922,0.305390,...,-0.109641,0.941353,0.588259,0.701469,-0.008119,0.609266,0.588271,0.705148,-0.149599,0.951764
2,Catch,0.317189,0.205269,-0.042855,0.999880,0.312762,0.183002,-0.030143,0.999924,0.312607,...,-0.078575,0.933145,0.591036,0.701711,-0.033041,0.615805,0.585116,0.706745,-0.108095,0.948470
3,Catch,0.329475,0.208577,-0.030689,0.999875,0.323808,0.185564,-0.017639,0.999923,0.323295,...,-0.085608,0.920749,0.588214,0.698682,-0.033107,0.607917,0.589124,0.706856,-0.112417,0.941997
4,Catch,0.339678,0.208752,-0.015762,0.999872,0.334384,0.185733,-0.001322,0.999920,0.333697,...,-0.085604,0.907931,0.590414,0.699538,-0.048863,0.607611,0.594268,0.706927,-0.115588,0.935928


In [23]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
333,Finish,0.150445,0.240923,0.087954,0.999955,0.142787,0.219081,0.111745,0.999961,0.142480,...,-0.230471,0.936479,0.642222,0.725503,-0.221023,0.669475,0.641296,0.761972,-0.321813,0.919642
334,Finish,0.150439,0.240932,0.087904,0.999955,0.142776,0.219088,0.111691,0.999961,0.142470,...,-0.230281,0.936687,0.642218,0.725474,-0.219843,0.670777,0.641314,0.761692,-0.321413,0.920052
335,Finish,0.150434,0.240941,0.087852,0.999955,0.142769,0.219096,0.111641,0.999961,0.142464,...,-0.230217,0.936948,0.642216,0.725418,-0.219457,0.672004,0.641330,0.761331,-0.321286,0.920452
336,Finish,0.150429,0.240950,0.087734,0.999955,0.142763,0.219102,0.111519,0.999961,0.142458,...,-0.230178,0.937167,0.642176,0.725418,-0.219441,0.673304,0.641334,0.761175,-0.321192,0.920842
337,Finish,0.150424,0.240960,0.087574,0.999955,0.142756,0.219109,0.111350,0.999961,0.142452,...,-0.229736,0.937346,0.642155,0.725406,-0.218169,0.674468,0.641334,0.760982,-0.320649,0.921261


In [24]:
cap = cv2.VideoCapture('FinishTrain.mp4')

with mp_pose.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, image = cap.read()
        
        
        #recolor
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #detections
        results = holistic.process(image)
        
        #recolor back to RGB
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                 , mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                 )
        
        #export coordinates

        pose = results.pose_landmarks.landmark #gets the pose landmarks
        pose_row = np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten()
            #gives each landmark component
            # .flatten() collapses an array into a single dimension, [[1,2],[3,4]] --> [1,2,3,4]

        #np.insert(pose_row, 0, class_name) #appends the class name to the CSV


            #Exports to CSV

       
            
        #get landmarks
        rshoulder = [pose[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,pose[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        rhip = [pose[mp_pose.PoseLandmark.RIGHT_HIP.value].x,pose[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
        rknee = [pose[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,pose[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            
            
        angle = calculate_angle(rshoulder, rhip, rknee) #calculate angles
            
        cv2.putText(image, str(angle), 
                        tuple(np.multiply(rhip, [640, 480]).astype(int)), #formats coords to vid
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                            )

            
            
            

        
        
        
        
        
        cv2.imshow('CatchFeed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
            
cap.release()
cv2.destroyAllWindows()

In [25]:
x=df.drop('class', axis=1) #features
y=df['class'] #target Value

In [26]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 1234)

In [27]:
X_train

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
189,0.150726,0.242842,0.080250,0.999948,0.143817,0.221075,0.103746,0.999958,0.143646,0.221708,...,-0.212649,0.937112,0.636411,0.730284,-0.220473,0.680839,0.638709,0.768339,-0.304824,0.931805
17,0.447105,0.239829,-0.093219,0.999206,0.444848,0.211459,-0.078283,0.998960,0.444412,0.210772,...,0.016193,0.895749,0.590222,0.677242,0.279415,0.568324,0.595567,0.713850,-0.020199,0.926665
299,0.150711,0.239634,0.100531,0.999956,0.143146,0.218296,0.124535,0.999964,0.142797,0.218904,...,-0.235040,0.934358,0.645949,0.724111,-0.228427,0.641714,0.643977,0.754575,-0.324947,0.910062
265,0.150791,0.239280,0.104632,0.999963,0.143384,0.218150,0.128822,0.999970,0.143037,0.218771,...,-0.247692,0.941824,0.637749,0.693289,-0.244043,0.664593,0.644249,0.751024,-0.337910,0.919937
69,0.125674,0.234777,0.059324,0.999013,0.119522,0.213890,0.081833,0.998889,0.120019,0.214389,...,-0.183677,0.945728,0.588639,0.682017,-0.115802,0.625667,0.596408,0.705429,-0.251883,0.945405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,0.150779,0.238956,0.089174,0.999965,0.143686,0.218173,0.112744,0.999971,0.143383,0.218811,...,-0.221733,0.940548,0.638048,0.730726,-0.216696,0.692411,0.638863,0.765680,-0.312026,0.931468
53,0.280007,0.176465,-0.049604,0.996535,0.273002,0.152866,-0.035479,0.995579,0.272084,0.152431,...,-0.020047,0.945974,0.588929,0.700151,0.120306,0.482600,0.596663,0.713855,-0.056696,0.935229
294,0.150734,0.239572,0.102507,0.999957,0.143191,0.218268,0.126582,0.999966,0.142844,0.218880,...,-0.236130,0.935618,0.646008,0.723705,-0.229097,0.647368,0.644042,0.753543,-0.325986,0.911854
211,0.150801,0.238824,0.092000,0.999967,0.143663,0.218037,0.115730,0.999973,0.143345,0.218675,...,-0.224834,0.940375,0.636126,0.704519,-0.218276,0.687813,0.639519,0.765195,-0.314759,0.929890


In [28]:
from sklearn.pipeline import make_pipeline #Makes an ML pipeline
from sklearn.preprocessing import StandardScaler # pre-procecessor, normalizes the data

from sklearn.linear_model import LogisticRegression, RidgeClassifier #different classification algorithms
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [29]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [30]:
shoulder = [pose[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,pose[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [pose[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,pose[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [pose[mp_pose.PoseLandmark.LEFT_WRIST.value].x,pose[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

rshoulder = [pose[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,pose[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
rhip = [pose[mp_pose.PoseLandmark.RIGHT_HIP.value].x,pose[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
rknee = [pose[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,pose[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]

In [31]:
rshoulder, rknee, rhip, shoulder, elbow, wrist

([0.08350507915019989, 0.3960346281528473],
 [0.3849518895149231, 0.713851809501648],
 [0.21327440440654755, 0.6708489060401917],
 [0.13373805582523346, 0.3935571312904358],
 [0.1825266033411026, 0.5471845865249634],
 [0.21392126381397247, 0.47174838185310364])

In [32]:
calculate_angle(shoulder, elbow, wrist)

40.21450763022275

In [46]:
cap = cv2.VideoCapture('ROWTRAIN.mp4')
counter = 0
stage = ""

with mp_pose.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, image = cap.read()
        
        
        #recolor
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #detections
        results = holistic.process(image)
        
        #recolor back to RGB
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                 , mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                 )
        
    
        
        pose = results.pose_landmarks.landmark #gets the pose landmarks
        pose_row = np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten()
            
        #get landmarks
        rshoulder = [pose[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,pose[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        rhip = [pose[mp_pose.PoseLandmark.RIGHT_HIP.value].x,pose[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
        rknee = [pose[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,pose[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
        
        rankle = [pose[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,pose[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
        
        
            
            
        angle = calculate_angle(rshoulder, rhip, rknee) #calculate angles
        knee = calculate_angle(rhip, rknee, rankle) 
            
        cv2.putText(image, str(angle), 
                        tuple(np.multiply(rhip, [1920, 1080]).astype(int)), #formats coords to vid
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                            )
        
        cv2.putText(image, str(knee), 
                        tuple(np.multiply(rknee, [1920, 1080]).astype(int)), #formats coords to vid
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                            )
        
        #rowCounter
        
        if(angle > 110):
            stage = "Finish"
            
        if(angle < 110 and angle >35):
            if(stage=="Finish"):
                stage = "Recovery"
            elif(stage == "Catch"):
                stage = "Drive"
            
        if (angle < 35 and stage == "Recovery"):
            stage = "Catch"
            counter+=1
            
            
            
            
        cv2.rectangle(image, (0,0), (500,73), (245,150,16), -1)
        cv2.putText(image, 'Strokes', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(image, 'STAGE', (90,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (90,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        
        
        
        
        
        
        cv2.imshow('RawFeed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
            
           
cap.release()
cv2.destroyAllWindows()